# Confirm TensorFlow can see the GPU

Simply select "GPU" in the Accelerator drop-down in Notebook Settings (either through the Edit menu or the command palette at cmd/ctrl-shift-P).

In [ ]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
print('Device name: {}'.format(device_name))
if device_name != '/device:GPU:0':
    raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

# Setting up

In [ ]:
!pip install librosa
!pip install matplotlib
!pip install pandas
!pip install numpy
!sudo add-apt-repository ppa:djcj/hybrid -y
!sudo apt-get update -y
!sudo apt-get install ffmpeg -y

In [1]:
import os
import pandas as pd
import numpy as np
import librosa
import librosa.display
import librosa.output
import IPython.display as ipd
#import matplotlib.pyplot as plt
#%matplotlib inline
PATH = '../datasets' #'/root/datasets'

/home/adrianohrl/anaconda3/lib/python3.5/site-packages/numba/errors.py:104: UserWarning: Insufficiently recent colorama version found. Numba requires colorama >= 0.3.9
  warnings.warn(msg)


# Importing Dataset

In [ ]:
!rm -rf ./sample_data/*
!rm -rf /root/datasets
!pip install -U -q PyDrive
import os
import sys
from tqdm import tqdm
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# choose a local (colab) directory to store the data.
local_download_path = os.path.expanduser(os.path.join(PATH, 'raw'))
try:
    os.makedirs(local_download_path)
except: pass

# 2. Auto-iterate using the query syntax
#    https://developers.google.com/drive/v2/web/search-parameters
file_list = drive.ListFile({'q': "'1hNxLdzo8GyzwD6EGuJNdye6N1h0mcRlX' in parents"}).GetList()

with tqdm(total=len(file_list), file=sys.stdout) as pbar:
    for f in file_list:
        fname = os.path.join(local_download_path, f['title'])
        pbar.write('[title: %s, id: %s] downloading to %s' % (f['title'], f['id'], fname))
        pbar.update(1)
        f_ = drive.CreateFile({'id': f['id']})
        f_.GetContentFile(fname)

# choose a local (colab) directory to store the data.
local_download_path = os.path.expanduser(os.path.join(PATH, 'noise'))
try:
    os.makedirs(local_download_path)
except: pass

# 2. Auto-iterate using the query syntax
#    https://developers.google.com/drive/v2/web/search-parameters
file_list = drive.ListFile({'q': "'18WwnYEglonXHw7cHZnL_iYTrNAqaqliL' in parents"}).GetList()

with tqdm(total=len(file_list), file=sys.stdout) as pbar:
    for f in file_list:
        fname = os.path.join(local_download_path, f['title'])
        pbar.write('[title: %s, id: %s] downloading to %s' % (f['title'], f['id'], fname))
        pbar.update(1)
        f_ = drive.CreateFile({'id': f['id']})
        f_.GetContentFile(fname)

In [ ]:
PATH = '../datasets'
path = os.path.join(PATH, 'input')
counter = 0
for parent, dirs, files in os.walk(path):
    for file in files:
        ext = os.path.splitext(file)[1][1:]
        if not parent.endswith(ext):
            dest = os.path.join(parent, ext)
            if not os.path.exists(dest):
                os.mkdir(dest)
            src, dest = os.path.join(parent, file), os.path.join(dest, file)
            os.rename(src, dest)
            counter += 1
            print('Moved {} to {}'.format(src, dest))
print('Moved {} files.'.format(counter))

In [2]:
   
def stretch(data, r = 1.0):
    return librosa.effects.time_stretch(data, r)
    
def noise(x, **params):
    noise = np.zeros(x.shape)
    noise_type = params['type'] if 'type' in params else 'white'
    if noise_type == 'white':
        distribution = params['distribution'] if 'distribution' in params else 'gaussian'
        sigma = params['sigma'] if 'sigma' in params else 0.005
        if distribution == 'gaussian':
            noise = np.random.normal(scale = sigma, size = x.shape)
        elif distribution == 'uniform':
            noise = np.random.uniform(low = -sigma, high = sigma, size = x.shape)
    return x + noise

def plot(x, sr, title = 'Raw Data'):
    plt.figure(figsize = (14, 5))
    plt.title(title)
    librosa.display.waveplot(x, sr = sr)
    plt.show()

In [ ]:
PATH = '../datasets'
np.random.seed(42)
df = pd.DataFrame(columns = ['input', 'output'])
path = os.path.join(PATH, 'output')
noisy_path = os.path.join(PATH, 'input')
if not os.path.exists(noisy_path):
    os.mkdir(noisy_path)
counter = 0
SIGMA_LOW = 1e-3
SIGMA_HIGH = 5e-2
NOISE_PARAMS = {'type': 'white', 'distribution': 'gaussian', 'sigma': 0.002}
for parent, dirs, files in os.walk(path):
    for file in files:
        src = os.path.join(parent, file)
        x, sr = librosa.load(src)
        NOISE_PARAMS['sigma'] = np.random.uniform(SIGMA_LOW, SIGMA_HIGH)
        x = noise(x, **NOISE_PARAMS)
        dest = os.path.join(noisy_path, '{}-{}'.format('-'.join([str(v) for v in NOISE_PARAMS.values()]), file))
        try:
            librosa.output.write_wav(dest, x, sr)
        except:
            print('Leaving...')
            break
        df.loc[counter] = [dest, src]
        counter += 1
        print('Generated new noisy audio file and saved in {}!'.format(dest))
print('Generated {} new noisy audio files.'.format(counter))
df.to_csv(os.path.join(PATH, 'raw2noisy.csv'))

Generated new noisy audio file and saved in ../datasets/input/white-0.019352465823520765-gaussian-sr8kHz-art167a.wav!
Generated new noisy audio file and saved in ../datasets/input/white-0.03929963866122016-gaussian-sr8kHz-art150b.wav!
Generated new noisy audio file and saved in ../datasets/input/white-0.023068998935443025-gaussian-sr8kHz-dt044d.wav!
Generated new noisy audio file and saved in ../datasets/input/white-0.03934072838659249-gaussian-sr8kHz-art007l.wav!
Generated new noisy audio file and saved in ../datasets/input/white-0.01956501238003407-gaussian-sr8kHz-art160b.wav!
Generated new noisy audio file and saved in ../datasets/input/white-0.028920179739528012-gaussian-sr8kHz-art149-d.wav!
Generated new noisy audio file and saved in ../datasets/input/white-0.007553427298528436-gaussian-sr8kHz-dt072j.wav!
Generated new noisy audio file and saved in ../datasets/input/white-0.024169869220396233-gaussian-sr8kHz-art005o.wav!
Generated new noisy audio file and saved in ../datasets/inpu

In [ ]:
df.loc[0, 'input']

In [ ]:
input_file = df.loc[0, 'input']
x, sr = load(input_file)
ipd.Audio(x, rate = sr)

In [ ]:
output_file = df.loc[0, 'output']
y, sr = load(output_file)
ipd.Audio(y, rate = sr)

# Dataset Split

In [3]:
import os
from time import time
from tqdm import tqdm
import pandas as pd
import numpy as np
import librosa
import librosa.display
import librosa.output
import IPython.display as ipd
from sklearn.cross_validation import train_test_split
#import matplotlib.pyplot as plt
#%matplotlib inline
PATH = '../datasets'

In [4]:
N = 5000
TEST_SIZE = 0.3
SEED = 42
df = pd.read_csv(os.path.join(PATH, 'raw2noisy.csv'))
M = len(df)
df_train, df_test = train_test_split(df.head(M), test_size = TEST_SIZE, random_state = SEED)
df_train, df_test = df_train.reset_index(), df_test.reset_index()
M_train, M_test = len(df_train), len(df_test)
X_train, Y_train = np.zeros((M_train, N, 1)), np.zeros((M_train, N, 1))
for i, row in tqdm(df_train.head(M).iterrows()):
    X_train[i, :, 0], Y_train[i, :, 0] = librosa.load(row['input'])[0][:N], librosa.load(row['output'])[0][:N]
X_test, Y_test = np.zeros((M_test, N, 1)), np.zeros((M_test, N, 1))
for i, row in tqdm(df_test.head(M).iterrows()):
    X_test[i, :, 0], Y_test[i, :, 0] = librosa.load(row['input'])[0][:N], librosa.load(row['output'])[0][:N]

690it [07:56,  1.30it/s]
296it [04:24,  1.05it/s]


# Encoder-Decoder LSTM

In [9]:
from keras.models import Sequential
from keras.layers import LSTM, Dropout, RepeatVector, TimeDistributed, Dense
from keras.callbacks import EarlyStopping
from keras.optimizers import Adam

In [ ]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
    raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

In [10]:
INPUT_LENGTH = N
N_CELLS = 64
DROPOUT = 0.4
LEARNING_RATE = 0.0001
OPTIMIZER = 'adam'
LOSS = 'mse'
ACTIVATION = 'tanh'

model = Sequential()
model.add(LSTM(N_CELLS, activation = ACTIVATION, input_shape = (INPUT_LENGTH, 1), name = 'encoder'))
model.add(Dropout(DROPOUT))
model.add(RepeatVector(N))
model.add(LSTM(N_CELLS, activation = ACTIVATION, return_sequences = True, name = 'decoder'))
model.add(TimeDistributed(Dense(1)))
model.compile(optimizer = Adam(lr = LEARNING_RATE), loss = LOSS)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
encoder (LSTM)               (None, 64)                16896     
_________________________________________________________________
dropout_2 (Dropout)          (None, 64)                0         
_________________________________________________________________
repeat_vector_2 (RepeatVecto (None, 5000, 64)          0         
_________________________________________________________________
decoder (LSTM)               (None, 5000, 64)          33024     
_________________________________________________________________
time_distributed_2 (TimeDist (None, 5000, 1)           65        
Total params: 49,985
Trainable params: 49,985
Non-trainable params: 0
_________________________________________________________________


In [ ]:
EPOCHS = 20
BATCH_SIZE = 50
PATIENCE = 3
MIN_DELTA = 1e-2
callbacks = [
    EarlyStopping(
        monitor='val_loss', 
        min_delta = MIN_DELTA, 
        patience = PATIENCE, 
        verbose = 0, 
        mode = 'auto', 
        restore_best_weights = True
    )
]
history = model.fit(
    X_train, 
    Y_train,
    validation_data = (X_test, Y_test),
    callbacks = callbacks,
    epochs = EPOCHS,
    batch_size = BATCH_SIZE,
    verbose = 1
)

Train on 690 samples, validate on 296 samples
Epoch 1/20
690/690 [==============================] - 158s 229ms/step - loss: 0.0040 - val_loss: 0.0039
Epoch 2/20
690/690 [==============================] - 159s 231ms/step - loss: 0.0039 - val_loss: 0.0039
Epoch 3/20
690/690 [==============================] - 157s 228ms/step - loss: 0.0038 - val_loss: 0.0039
Epoch 4/20
200/690 [=======>......................] - ETA: 1:51 - loss: 0.0041

In [ ]:
ts = time()
df_report_filename = os.path.join(PATH, 'report.csv')
if not os.path.exists(report_df_filename):
    columns = [
        'timestamp', 
        'json', 'h5', 
        'n', 
        'm', 
        'test_size', 
        'seed', 
        'n_cells', 
        'dropout', 
        'optimizer', 
        'learning_rate', 
        'loss', 
        'activation',
        'epochs',
        'batch_size',
        'patience',
        'min_delta'
    ]
    df_report = pd.DataFrame()
else:
    df_report = pd.read_csv(report_df_filename)
json = model.to_json()
json_filename = os.path.join(PATH, 'model-{}.json'.format(int(ts)))
h5_filename = os.path.join(PATH, 'model-{}.h5'.format(int(ts)))
with open(json_filename, 'w') as file:
    file.write(json)
    file.close()
    model.save_weights(h5_filename)
    df_report.loc[len(df_report)] = [
        str(ts), 
        N, 
        M, 
        TEST_SIZE, 
        SEED, 
        N_CELLS, 
        DROPOUT, 
        OPTIMIZER, 
        LEARNING_RATE, 
        LOSS, 
        ACTIVATION,
        EPOCHS,
        BATCH_SIZE,
        PATIENCE,
        MIN_DELTA
    ]
    print('Saved model to disk.')

In [ ]:
with open(json_filename, 'r') as file:
    json = file.read()
    file.close()
    model = model_from_json(json)
    model.load_weights(h5_filename)
    print("Loaded model from disk")

In [ ]:
Y_pred = model.predict(X, verbose = 0)
save(os.path.join(path, 'pred.wav'), Y_pred.flatten(), sr)
ipd.Audio(Y_pred.flatten(), rate = sr)

In [ ]:
#train_mse = history.history['mse']
#validation_mse = history.history['val_mean_squared_error']
train_loss = history.history['loss']
#validation_loss = history.history['val_loss']

epochs = range(len(train_loss))

#plt.plot(epochs, train_mse, 'b-', label = 'Training MSE')
#plt.plot(epochs, validation_mse, 'r-', label = 'Validation MSE')
#plt.title('Training and validation MSE')
#plt.legend()

plt.figure()
plt.plot(epochs, train_loss, 'b-', label = 'Training loss')
#plt.plot(epochs, validation_loss, 'r-', label = 'Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

In [ ]:
fig = plt.figure(figsize = (14, 5))
librosa.display.waveplot(Y_pred.flatten(), sr)